In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://aifactory.space/competition/detail/2226

# Git

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/AI_SPARK/AI_SPARK_Challenge_2023

/content/drive/MyDrive/Colab Notebooks/AI_SPARK/AI_SPARK_Challenge_2023


브랜치
- master
- ksw_main

In [ ]:
!git branch ksw_main

In [ ]:
!git checkout master

Switched to branch 'master'
Your branch is up to date with 'origin/master'.


In [ ]:
!git remote update


Fetching origin
Already on 'ksw_main'


In [ ]:
!git checkout ksw_main

Switched to branch 'ksw_main'


In [ ]:
!git add -all

In [ ]:
!git branch

  ksw_main
* master


# Train

## Import

In [2]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184112 sha256=c420c7edc72fca9ff8113c8caf2de1b54694a7dd75cde27979bd9b71b14e6061
  Stored in directory: /root/.cache/pip/wheels/1b/9c/b8/9759d7cc64a1e01bb9872ade80cb7db445ccf506e083325106
Successfully built pyod


In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import *

from pyod.models.abod import ABOD
from sklearn.ensemble import IsolationForest

import tensorflow as tf
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.backend import clear_session

import pickle
import numpy as np


## Data Load

In [4]:
df_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AI_SPARK/Data/train_data.csv')

In [5]:
def tyep_to_hp(type_no):
  if type_no == 1:
    return 20
  elif type_no == 2:
    return 10
  elif type_no == 3:
    return 50
  else:
    return 30


df_data = df_data.drop(axis=1, columns=['out_pressure'])
df_data['type'] = df_data['type'].apply(tyep_to_hp)


In [6]:
df_data

,air_inflow,air_end_temp,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,1.59,41.00,20.53,1680.0,58.67,2.93,30
1,2.97,59.28,38.40,3142.0,74.91,3.75,30
2,1.91,45.29,24.73,2023.0,62.48,3.12,30
3,2.37,51.33,30.63,2506.0,67.84,3.39,30
4,1.90,45.21,24.65,2017.0,62.41,3.12,30
...,...,...,...,...,...,...,...
2458,2.28,50.20,29.53,2416.0,66.84,3.34,30
2459,2.04,46.94,26.34,2155.0,63.94,3.20,30
2460,1.19,35.74,15.39,1259.0,53.99,2.70,30
2461,1.21,36.00,15.64,1280.0,54.22,2.71,30


## AutoEncoder

In [39]:
clear_session()

n_inputs = df_data.shape[1]

# configure
input_data = Input(shape=(n_inputs,))

# layers
encoded = Dense(n_inputs//2, activation='relu')(input_data)
dropout = Dropout(0.2)(encoded)
encoded = Dense(n_inputs//4, activation='relu')(encoded)
dropout = Dropout(0.2)(encoded)

n_bottneck = 200
bottneck = Dense(n_bottneck, activation='relu')(encoded)

decoded = Dense(n_inputs//4, activation='relu')(bottneck)
dropout = Dropout(0.2)(decoded)
decoded = Dense(n_inputs//2, activation='relu')(decoded)
dropout = Dropout(0.2)(decoded)
decoded = Dense(n_inputs, activation='relu')(decoded)

# Models
autoencoder = Model(input_data, decoded) # autoencoder

encoder = Model(input_data, encoded) # encoder

In [40]:
autoencoder.compile(optimizer='adam', loss='mse')

In [41]:
es = EarlyStopping(monitor = 'val_loss',patience=5,restore_best_weights=True, verbose=1)

In [42]:
scaler = MinMaxScaler()
train_df = scaler.fit_transform(df_data)

In [43]:
x_train, x_val, _, _ = train_test_split(train_df,train_df,random_state=2023,test_size=0.3)

autoencoder.fit(x_train, x_train, epochs = 100, callbacks = [es], validation_split = 0.3)

Epoch 1/100
38/38 [==============================] - 2s 10ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 2/100
38/38 [==============================] - 0s 5ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 3/100
38/38 [==============================] - 0s 5ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 4/100
38/38 [==============================] - 0s 4ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 5/100
38/38 [==============================] - 0s 5ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 6/100
38/38 [==============================] - 0s 5ms/step - loss: 0.2127 - val_loss: 0.2298
Epoch 6: early stopping


In [44]:
reconstructions = autoencoder.predict(x_val)
train_loss = tf.keras.losses.mae(reconstructions, x_val)

24/24 [==============================] - 0s 3ms/step


In [45]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

Threshold:  0.5822106323619227


In [46]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  pred = tf.math.less(loss, threshold)
  pred = np.where(pred,0,1)
  return pred

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))



In [47]:
preds = predict(autoencoder, x_val, threshold)
print_stats(preds, np.array([0]*x_val.shape[0]))

Accuracy = 0.8267929634641408


## KFold

In [ ]:
kf = KFold(random_state=2023,shuffle=True)

In [ ]:
for i, (train_index, test_index) in enumerate(kf.split(df_data)):
  print('='*10)
  print(i)
  print('='*10)
  x_train = df_data.loc[train_index]
  x_test = df_data.loc[test_index]

  scaler = MinMaxScaler()
  x_train = scaler.fit_transform(x_train)
  x_test = scaler.transform(x_test)

  

0
Epoch 1/100
44/44 [==============================] - 3s 9ms/step - loss: 0.1595 - val_loss: 0.2145
Epoch 2/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1369 - val_loss: 0.1875
Epoch 3/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1183 - val_loss: 0.1649
Epoch 4/100
44/44 [==============================] - 0s 7ms/step - loss: 0.1031 - val_loss: 0.1459
Epoch 5/100
44/44 [==============================] - 0s 5ms/step - loss: 0.0909 - val_loss: 0.1306
Epoch 6/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0810 - val_loss: 0.1181
Epoch 7/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0732 - val_loss: 0.1080
Epoch 8/100
44/44 [==============================] - 0s 7ms/step - loss: 0.0669 - val_loss: 0.1001
Epoch 9/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0619 - val_loss: 0.0937
Epoch 10/100
44/44 [==============================] - 0s 6ms/step - loss: 0.0581 - val_loss: 0.0888
Epoch 1